In [ ]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
start_time = datetime.now()

date = datetime.now().date()

filepath_novinki = r"R:\Общая информация блока логистики\Ввод новинок\Прочие запросы\PBL\PBL новинки\Новинки к рассмотрению на PBL.xlsx"
filepath_inn = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\КА ИНН.xlsx"
filepath_tovari = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\Товары для ПЦ.xlsx"
filepath_mp = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\!Для новинок\Приходы КА на ГМ(МП)_ фильтр месяц.xlsx"
filepath_okrug = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\РЦ - Округ.xlsx"
filepath_mias = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\!Для новинок\МИАС.xlsx"
filepath_rollout = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - General\Операционные требования\Статус ролл-аута по опер.требованиям на 2025.xlsx"
filepath_iscl = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\!Для новинок\!!!!СПИСОК ИСКЛЮЧЕНИЙ.xlsx"
filepath_iscl_mp = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\КА с МП на РЦ согласованные исключения.xlsx"

novinki = pd.read_excel(filepath_novinki, sheet_name="Новинки")
inn_df = pd.read_excel(filepath_inn, sheet_name="КА ИНН")
tovari_df = pd.read_excel(filepath_tovari, sheet_name="СПР_ЛОГИСТИЧЕСКИЕ КАТЕГОРИИ")
mp = pd.read_excel(filepath_mp)
okrug = pd.read_excel(filepath_okrug)
mias = pd.read_excel(filepath_mias)
rollout = pd.read_excel(filepath_rollout, sheet_name="ТОП-250+ на 2023")
iscl = pd.read_excel(filepath_iscl)
iscl_mp = pd.read_excel(filepath_iscl_mp)

novinki = novinki.query("`Дата внесения` == @date")
novinki = novinki[['Дата внесения','Неделя внесения', 'Дата закрытия', 'ФИО вносившего', 'РЦ',
       'Сц (РЦ+ТП)', 'Код ТП ввода', 'ТП ввод', 'Форматы', 'ТГ 20 ввод',
       'ТГ 21 ввод', 'ТГ 22 ввод', 'Код КА', 'Наименование КА', 'Тип склада',
       'Комментарий_лог. Параметры', 'Комментарий КИП',
       'Комментарий менеджера по технологии размещения ассортимента',
       'Комментарий_лог.контракты', 'Код РЦ', 'Сц СУАГ']]

def fillna_str(df, cols):
    """Заменяет NaN на пустую строку и приводит к str."""
    df[cols] = df[cols].fillna('').astype(str)
    return df

def create_key(df, new_col, cols):
    """Создаёт ключ по списку колонок путем их конкатенации."""
    df[new_col] = df[cols].fillna('').astype(str).agg(''.join, axis=1)
    return df

def clean_int_str(val):
    """Если строка состоит из цифр — приводит к int, иначе остаётся как есть."""
    if isinstance(val, str) and val.isdigit():
        return int(val)
    return val

def compute_itog(row):
    """Логика расчёта итогового статуса по строке."""
    rc = row['РЦ']
    rc_mp = row['РЦ/МП']
    si = row['СИ/НЕ СИ']
    tender = row['Тендер/ Не Тендер']
    uk = row['Унифицированный комментарий']
    tp = row.get('Тип поставки', '')

    if rc_mp == 'МП':
        return 'МП'
    if si == 'Собственный импорт' or tender == 'Тендерная позиция':
        return 'КА исключение (СИ/Тендер)'

    in_special_rc = rc in ["РЦ Иваново", "РЦ Подольск Федеральный (а)"]
    if in_special_rc:
        if uk == 'Подписал':
            return 'Подписал ДС (допустимо)' if tp == 'ФД' else 'Не подписал ДС (ФРЦ)'
        else:
            return 'Не подписал ДС (ОТ)'

    # Для всех остальных
    return 'Подписал ДС (допустимо)' if uk == 'Подписал' else 'Не подписал ДС (ОТ)'


# novinki, iscl, iscl_mp, inn_df, tovari_df, mp, okrug, mias, rollout

fillna_str(novinki, ["Код ТП ввода", "Код КА"])
fillna_str(iscl, ["Код ТП ввода"])
fillna_str(inn_df, ["Контрагент Код", "Контрагент ИНН"])
fillna_str(tovari_df, ["Товар Код"])
fillna_str(mp, ['Контрагент ИНН', 'РЦ по основному складу Наименование', 'Лог категория ID'])
fillna_str(iscl_mp, ['ИНН КА', 'РЦ', 'Код товара'])


inn_df = inn_df.rename(columns={'Контрагент Код': 'Код КА'})
tovari_df = tovari_df.rename(columns={'Товар Код': 'Код ТП ввода'})
okrug = okrug.rename(columns={'Полное наименование':'РЦ'})


create_key(iscl_mp, 'СЦ4', ['ИНН КА', 'РЦ', 'Код товара'])
create_key(iscl_mp, 'СЦ5', ['ИНН КА', 'Код товара'])
create_key(mp, 'СЦ1', ['Контрагент ИНН', 'РЦ по основному складу Наименование', 'Лог категория ID'])


mias['Код КА'] = mias['Код КА'].apply(clean_int_str)
mias_vpr = mias.query('`Состояние ПЦ` == "Активный" & Формат == "БФ"').copy()
mias_vpr = mias_vpr[['Код КА','Лог. категория','Регион(округ)','РЦ']]

mias_sc2 = mias_vpr.dropna()
create_key(mias_sc2, 'СЦ2', ['Код КА','РЦ','Лог. категория'])
mias_sc2 = mias_sc2.drop_duplicates(subset=['СЦ2'])

mias_sc3 = mias_vpr[mias_vpr['РЦ'].isna() | (mias_vpr['РЦ'] == 'NaN')].copy()
create_key(mias_sc3, 'СЦ3', ['Код КА','Регион(округ)','Лог. категория'])
mias_sc3 = mias_sc3.drop_duplicates(subset=['СЦ3'])

df_fed = mias.query('`Состояние ПЦ` == "Активный" & `Тип поставки` == "ФД"').copy()
create_key(df_fed, 'СЦ0', ['Код КА','Лог. категория'])
df_fed = df_fed.drop_duplicates(subset=['СЦ0'])

rollout = rollout.query('`Унифицированный комментарий` == "Подписал"') \
                 [['Код КА', 'Унифицированный комментарий']]
fillna_str(rollout, ['Код КА'])


novinki = novinki.merge(inn_df[['Код КА','Контрагент ИНН']], on='Код КА', how='left')
novinki = novinki.merge(tovari_df[['Код ТП ввода','Лог категория ID','Собственный импорт DESC','Признак тендерной позиции Наименование']],
                        on='Код ТП ввода', how='left')
novinki = novinki.merge(okrug, on='РЦ', how='left')
novinki = novinki.rename(columns={
    'Контрагент ИНН': 'ИНН КА',
    'Лог категория ID': 'ЛК',
    'Собственный импорт DESC': 'СИ/НЕ СИ',
    'Признак тендерной позиции Наименование': 'Тендер/ Не Тендер'
})

novinki['ИНН КА'] = novinki['ИНН КА'].apply(lambda x: str(int(float(x))) if str(x).replace('.','').isdigit() else str(x))

create_key(novinki, 'СЦ0', ['Код КА','ЛК'])
create_key(novinki, 'СЦ1', ['ИНН КА','РЦ','ЛК'])
create_key(novinki, 'СЦ2', ['Код КА','РЦ','ЛК'])
create_key(novinki, 'СЦ3', ['Код КА','Округ','ЛК'])
create_key(novinki, 'СЦ4', ['ИНН КА','РЦ','Код ТП ввода'])
create_key(novinki, 'СЦ5', ['ИНН КА','Код ТП ввода'])

novinki = novinki.merge(rollout, on='Код КА', how='left')
novinki = novinki.merge(df_fed[['СЦ0','Тип поставки']], on='СЦ0', how='left')
novinki = novinki.merge(mp[['СЦ1','Контрагент Название']], on='СЦ1', how='left')
novinki = novinki.merge(mias_sc2[['СЦ2','Код КА']], on='СЦ2', how='left')
novinki = novinki.merge(mias_sc3[['СЦ3','Код КА']], on='СЦ3', how='left')
novinki = novinki.merge(iscl, on='Код ТП ввода', how='left')

mask = novinki['РЦ'] == 'сеть РЦ'
novinki.loc[mask, ['Комментарий','Код товара']] = (
    novinki[mask]
    .merge(iscl_mp[['СЦ4','Комментарий','Код товара']], on='СЦ4', how='left')[['Комментарий','Код товара']].values
)
novinki.loc[~mask, ['Комментарий','Код товара']] = (
    novinki[~mask]
    .merge(iscl_mp[['СЦ5','Комментарий','Код товара']], on='СЦ5', how='left')[['Комментарий','Код товара']].values
)

novinki.loc[novinki['Наименование позиции'].notna(), 'Тендер/ Не Тендер'] = "Не тендерная позиция"

novinki['РЦ/МП'] = np.nan
mask_mp = novinki['Код товара'].notna() & (novinki['Комментарий'] != 'Проект ДС')
novinki.loc[mask_mp | (novinki['Форматы'] == 'ДС'), 'РЦ/МП'] = 'РЦ'

mask = novinki['РЦ/МП'].isna()
novinki.loc[mask, 'РЦ/МП'] = np.where(
    novinki.loc[mask, 'Контрагент Название'].notna() &
    novinki.loc[mask, 'Код КА_y'].isna() &
    novinki.loc[mask, 'Код КА'].isna(),
    'МП', 'РЦ'
)

novinki['Итог по КА'] = novinki.apply(compute_itog, axis=1)

drop_cols = [
    'ЛК','Наименование позиции','Округ',
    'СЦ0','СЦ1','СЦ2','СЦ3','Унифицированный комментарий','Тип поставки',
    'Код КА_y','Код КА'
]
novinki = (novinki
           .drop(drop_cols, axis=1)
           .rename(columns={'Код КА_x': 'Код КА','Контрагент Название': 'КА МП на ГМ'}))

novinki['Дата внесения'] = novinki['Дата внесения'].dt.strftime('%d.%m.%Y')

final_cols = [
    'Дата внесения','Неделя внесения','Дата закрытия',
    'ФИО вносившего','РЦ','Сц (РЦ+ТП)','Код ТП ввода','ТП ввод',
    'Форматы','ТГ 20 ввод','ТГ 21 ввод','ТГ 22 ввод',
    'Код КА','Наименование КА','Тип склада',
    'Комментарий_лог. Параметры','Комментарий КИП',
    'Комментарий менеджера по технологии размещения ассортимента',
    'Комментарий_лог.контракты','Код РЦ','Сц СУАГ',
    'ИНН КА','КА МП на ГМ','РЦ/МП','СИ/НЕ СИ',
    'Тендер/ Не Тендер','Итог по КА'
]
novinki = novinki[final_cols]

desktop_path = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\!Для новинок\Новинки к рассмотрению на PBL.xlsx"

# Создаем объект ExcelWriter с использованием xlsxwriter
with pd.ExcelWriter(desktop_path, engine='xlsxwriter') as writer:
    novinki.to_excel(writer, index=False, sheet_name='Новинки')

    # Получаем объект текущего листа
    worksheet = writer.sheets['Новинки']

    # Шрифт и размер по всей таблице
    general_format = writer.book.add_format({'font_name': 'Calibri', 'font_size': 9})

    # Применяем шрифт ко всем ячейкам таблицы
    worksheet.set_column('A:Z', None, general_format)  # Применяем ко всем столбцам

    # Стили для заголовков
    gray_fill = writer.book.add_format({'bg_color': '#D3D3D3', 'font_name': 'Calibri', 'font_size': 9, 'bold': True, 'text_wrap': True, 'border': 1, 'align': 'top', 'valign': 'top'})
    maroon_fill = writer.book.add_format({'bg_color': '#F8CBAD', 'font_name': 'Calibri', 'font_size': 9, 'bold': True, 'text_wrap': True, 'border': 1, 'align': 'top', 'valign': 'top'})
    yellow_fill = writer.book.add_format({'bg_color': '#FFFF00', 'font_name': 'Calibri', 'font_size': 9, 'bold': True, 'text_wrap': True, 'border': 1, 'align': 'top', 'valign': 'top'})
    green_fill = writer.book.add_format({'bg_color': '#C6E0B4', 'font_name': 'Calibri', 'font_size': 9, 'bold': True, 'text_wrap': True, 'border': 1, 'align': 'top', 'valign': 'top'})
    orange_fill = writer.book.add_format({'bg_color': '#FFC000', 'font_name': 'Calibri', 'font_size': 9, 'bold': True, 'text_wrap': True, 'border': 1, 'align': 'top', 'valign': 'top'})
    blue_fill = writer.book.add_format({'bg_color': '#BDD7EE', 'font_name': 'Calibri', 'font_size': 9, 'bold': True, 'text_wrap': True, 'border': 1, 'align': 'top', 'valign': 'top'})
    
    # Стили для заголовков таблицы (первая строка)
    header_range = list(novinki.columns)
    
    # Применяем стили к шапке таблицы
    worksheet.set_row(0, None, general_format)  # Применяем шрифт ко всем заголовкам

    # Применяем разные цвета и перенос текста к заголовкам
    worksheet.write(0, 0, header_range[0], gray_fill)  # 'Дата внесения'
    worksheet.write(0, 1, header_range[1], gray_fill)  # 'Неделя внесения'
    worksheet.write(0, 2, header_range[2], gray_fill)  # 'Дата закрытия'
    worksheet.write(0, 3, header_range[3], gray_fill)  # 'ФИО вносившего'
    worksheet.write(0, 4, header_range[4], gray_fill)  # 'РЦ'

    worksheet.write(0, 5, header_range[5], maroon_fill)  # 'Сц (РЦ+ТП)'

    worksheet.write(0, 6, header_range[6], gray_fill)  # 'Код ТП ввода'
    worksheet.write(0, 7, header_range[7], gray_fill)  # 'ТП ввод'
    worksheet.write(0, 8, header_range[8], gray_fill)  # 'Форматы'
    worksheet.write(0, 9, header_range[9], gray_fill)  # 'ТГ 20 ввод'
    worksheet.write(0, 10, header_range[10], gray_fill)  # 'ТГ 21 ввод'
    worksheet.write(0, 11, header_range[11], gray_fill)  # 'ТГ 22 ввод'
    worksheet.write(0, 12, header_range[12], gray_fill)  # 'Код КА'
    worksheet.write(0, 13, header_range[13], gray_fill)  # 'Наименование КА'
    worksheet.write(0, 14, header_range[14], gray_fill)  # 'Тип склада'

    worksheet.write(0, 15, header_range[15], yellow_fill)  # 'Комментарий_лог. Параметры'
    worksheet.write(0, 16, header_range[16], yellow_fill)  # 'Комментарий КИП'
    worksheet.write(0, 17, header_range[17], yellow_fill)  # 'Комментарий менеджера по технологии размещения ассортимента'
    worksheet.write(0, 18, header_range[18], yellow_fill)  # 'Комментарий_лог.контракты'

    worksheet.write(0, 19, header_range[19], gray_fill)  # 'Код РЦ'
    worksheet.write(0, 20, header_range[20], gray_fill)  # 'Сц СУАГ'
    worksheet.write(0, 21, header_range[21], gray_fill)  # 'ИНН КА'
    worksheet.write(0, 22, header_range[22], gray_fill)  # 'КА МП на ГМ'

    worksheet.write(0, 23, header_range[23], green_fill)  # 'РЦ/МП'
    worksheet.write(0, 24, header_range[24], green_fill)  # 'СИ/НЕ СИ'
    worksheet.write(0, 25, header_range[25], green_fill)  # 'Тендер/ Не Тендер'

    worksheet.write(0, 26, header_range[26], orange_fill)  # 'Итог по КА'
    
    worksheet.set_column('A:A', None, general_format)
    worksheet.set_column(15, 15, 12, general_format)
    worksheet.set_column(16, 16, 12, general_format)
    worksheet.set_column(17, 17, 20, general_format)
    worksheet.set_column(18, 18, 12, general_format)
    worksheet.set_column(26, 27, 10, general_format)
    
print(f"Файл сохранен в {desktop_path}")

end_time = datetime.now()
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time}")